In [ ]:
SELECT  FORMAT_DATE("%Y-%m-%d",order_placed_timestamp) AS Date,
        CASE 
           WHEN bh.is_marketplace=true THEN "Marketplace"
          ELSE "Retail"
        END AS SellerType,
      ROUND(SUM(order_item_sales),2) AS GMV
    FROM `tal-production-data-bi.tal_dm_order.orders_2020_2030` o,
    UNNEST(o.order_items) oi
    INNER JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
               ON bh.product_id = oi.product_id
    WHERE   
        DATE(order_placed_timestamp) = DATE_ADD(CURRENT_DATE(), INTERVAL -1 DAY) 
        AND oi.is_authed_not_return_cancel IS TRUE 
        AND oi.order_item_status NOT IN ('Return Canceled','Canceled')
        #AND oi.order_item_status LIKE 'Shipped'
        AND order_authorisation_status IN ('Auth')
        AND is_returned = false
        AND is_cancelled = false
        AND is_voucher = false
        AND is_test_customer = false
GROUP BY SellerType, Date 